# Pré-processamentos

Definimos no nosso relatórios algumas estratégias mais comuns para lidar com os problemas desse dataset. Aqui apliquei normalização das strings, conversão da data de nascimento em idade e imputação dos valores ausentes da data de nascimento pelo método IterativeImputer do Sklearn, além disso, usamos os valores mais comuns associados à categorias com alta correlação para imputar valores ausentes nas colunas DS_INDICACAO_ACIDENTE e DS_TIPO_ATENDIMENTO.
Consideramos que os valores nulos de QT_DIA_SOLICITADO corresponde a não solicitação de dias de internação, por isso trocamos os valos NaN por zero.


_ by Lisandra Moura _


# Remoção das colunas


>>> NR_SEQ_REQUISICAO,
>>> NR_SEQ_ITEM,
>>> NR_PRODUTO,
>>> DS_TIPO_CONSULTA,
>>> QT_TEMPO_DOENCA,
>>> DS_UNIDADE_TEMPO_DOENCA,
>>> DS_TIPO_DOENCA,
>>> DS_TIPO_SAIDA,
>>> DS_TIPO_INTERNACAO (* TENTAR ALGUNS PRÉ-PROCESSAMENTOS ANTES),
>>> DS_REGIME_INTERNACAO (*),
>>> DS_TIPO_ACOMODACAO,
>>> CD_GUIA_REFERENCIA,
>>> DS_INDICACAO_CLINICA (* INFORMAÇÕES QUE PODEM CONTRIBUIR PARA IMPUTAR INFORMAÇÕES),
>>> CD_ITEM,
>>> DS_ITEM,
>>> DS_CLASSE,
>>> DS_SUBGRUPO,
>>> DS_GRUPO.

In [43]:
import pandas as pd
import numpy as np
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from unidecode import unidecode
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
import datetime
import jdcal

In [44]:
df = pd.read_csv("C:/Users/moura/Projetos/KDD_TeamOne/data/classificacao_binaria_train.csv", 
                 encoding='ISO-8859-1', sep=';')

#df = pd.read_csv("C:/Users/moura/Projetos/KDD_TeamOne/data/classificacao_binaria_test.csv", 
#                 encoding='ISO-8859-1', sep=';')

def julian_to_age(julian_date):
    # Convertendo a data juliana para data gregoriana
    year, month, day = jdcal.jd2gcal(0, julian_date)[:3]
    date_gregorian = datetime.date(year, month, day)
    
    # Calculando a idade
    today = datetime.date.today()
    age = today.year - date_gregorian.year - ((today.month, today.day) < (date_gregorian.month, date_gregorian.day))
    return age


# Removendo as colunas
columns_to_drop = [
        'NR_SEQ_REQUISICAO', 'DT_REQUISICAO' , 'NR_SEQ_ITEM','NR_PRODUTO', 'DS_TIPO_CONSULTA',
        'QT_TEMPO_DOENCA','DS_UNIDADE_TEMPO_DOENCA', 'DS_TIPO_DOENCA','DS_TIPO_SAIDA',
        'DS_TIPO_INTERNACAO', 'DS_REGIME_INTERNACAO', 'DS_TIPO_ACOMODACAO','CD_GUIA_REFERENCIA',
        #'DS_INDICACAO_CLINICA',
        'CD_ITEM', 'DS_ITEM', 'DS_CLASSE', 'DS_SUBGRUPO', 'DS_GRUPO'
]

df = df.drop(columns=columns_to_drop, errors='ignore')

# Removendo acentos das colunas str

def applyunicode(df, columns):
        for col in columns:
                df[col] = df[col].apply(lambda x: unidecode(str(x)) if pd.notnull(x) else x)
        return df

object_cols = df.select_dtypes(include='object').columns
df = applyunicode(df,object_cols)

# Substituindo os valores incorretos manualmente

df['DS_TIPO_GUIA'] = df['DS_TIPO_GUIA'].str.replace('solicitac?o', 'solicitacao')
df['DS_TIPO_GUIA'] = df['DS_TIPO_GUIA'].str.replace('prorrogac?o', 'PRORROGACAO')
df['DS_TIPO_GUIA'] = df['DS_TIPO_GUIA'].str.replace('internac?o', 'INTERNACAO')	
df['DS_INDICACAO_ACIDENTE'] = df['DS_INDICACAO_ACIDENTE'].str.replace('N?o', 'Nao')
df['DS_TIPO_ATENDIMENTO'] = df['DS_TIPO_ATENDIMENTO'].str.replace('Internac?o', 'INTERNACAO')
df['DS_TIPO_ATENDIMENTO'] = df['DS_TIPO_ATENDIMENTO'].str.replace('Remoc?o', 'REMOCAO')

# Colocando todos os textos das colunas str em caps look

for col in df.select_dtypes(include='object').columns:
    df[col] = df[col].str.upper()
    

# Imputação interativa nos valores nulos
imputer = IterativeImputer()
df['DT_NASCIMENTO'] = imputer.fit_transform(df[['DT_NASCIMENTO']])

# Convertendo formato juliano em idade
df['IDADE'] = df['DT_NASCIMENTO'].apply(julian_to_age)


# Usando valores mais comuns associdados à categoria com correlação
# DS_INDICACAO_ACIDENTE correlação com DS_TIPO_PREST_SOLICITANTE
most_common_accidents = df.groupby('DS_TIPO_PREST_SOLICITANTE')['DS_INDICACAO_ACIDENTE'].apply(lambda x: x.mode()[0])

for prestador in most_common_accidents.index:
    df.loc[(df['DS_TIPO_PREST_SOLICITANTE'] == prestador) & (df['DS_INDICACAO_ACIDENTE'].isna()), 'DS_INDICACAO_ACIDENTE'] = most_common_accidents[prestador]

# Usando valores mais comuns associdados à categoria com correlação
# DS_TIPO_ATENDIMENTO correlação com DS_TIPO_GUIA

most_common_atendimentos = df.groupby('DS_TIPO_GUIA')['DS_TIPO_ATENDIMENTO'].apply(lambda x: x.mode()[0])
for guia in most_common_atendimentos.index:
    df.loc[(df['DS_TIPO_GUIA'] == guia) & (df['DS_TIPO_ATENDIMENTO'].isna()), 'DS_TIPO_ATENDIMENTO']= most_common_atendimentos[guia]

#Nas colunas QT_DIA_SOLICITADO substituimos os valores nulos por zero

df['QT_DIA_SOLICITADO'] = df['QT_DIA_SOLICITADO'].fillna(0)


# Até aqui tiramos a maioria dos nulos, mas ainda precisamos resolver o problema do CD_CID

In [45]:
nulos = df['CD_CID'].isna().sum()
print("Temos nos dados originais o total de", nulos, " nulos.")

Temos nos dados originais o total de 235294  nulos.


Em CD_CID temos um grande problema pois, temos as informações em DS_INDICACAO de alguma doenças, como se pode ver no exemplo a baixo.
Porém, são informações difusas e sem padrão.
A minha ideia é imputar os dados que já temos em DS_INDICACAO_CLINICA em CD_CID e depois usar K-means para imputar o restante, utilizando tecnicas de agrupamentos e correlações como feito colunas anteriores. 

# Imputando dados manualmente

In [46]:
#Você pode visualizar os valos que temos em DS e que não temos em CD_CID 

visualizacao = df.loc[(df['DS_INDICACAO_CLINICA'] == 'C50') & (df['CD_CID'].isna())]
#visualizacao = df.loc[(df['DS_INDICACAO_CLINICA'] == 'GLAUCOMA') & (df['CD_CID'].isna())]
#visualizacao = df.loc[df['DS_INDICACAO_CLINICA'] == 'CD DE MAMA']

visualizacao

,DS_TIPO_GUIA,DT_NASCIMENTO,DS_TIPO_PREST_SOLICITANTE,DS_CBO,DS_INDICACAO_ACIDENTE,DS_CARATER_ATENDIMENTO,QT_DIA_SOLICITADO,DS_TIPO_ATENDIMENTO,CD_CID,DS_INDICACAO_CLINICA,DS_TIPO_ITEM,QT_SOLICITADA,DS_STATUS_ITEM,IDADE
2835,GUIA DE SOLICITACAO SP/SADT,2432021.0,CLINICA,MEDICO ONCOLOGISTA CLINICO,NAO ACIDENTE,ELETIVA,0.0,QUIMIOTERAPIA,NaN,C50,PROCEDIMENTOS,1.0,AUTORIZADO,77
2843,GUIA DE SOLICITACAO SP/SADT,2437426.0,CLINICA,MEDICO ONCOLOGISTA CLINICO,NAO ACIDENTE,ELETIVA,0.0,QUIMIOTERAPIA,NaN,C50,PROCEDIMENTOS,1.0,NEGADO,62
7384,GUIA DE SOLICITACAO SP/SADT,2437426.0,CLINICA,MEDICO ONCOLOGISTA CLINICO,NAO ACIDENTE,ELETIVA,0.0,QUIMIOTERAPIA,NaN,C50,PROCEDIMENTOS,2.0,NEGADO,62
9005,GUIA DE SOLICITACAO SP/SADT,2444370.0,CLINICA,MEDICO ONCOLOGISTA CLINICO,NAO ACIDENTE,ELETIVA,0.0,QUIMIOTERAPIA,NaN,C50,PROCEDIMENTOS,1.0,AUTORIZADO,43
17587,GUIA DE SOLICITACAO SP/SADT,2439288.0,CLINICA,MEDICO ONCOLOGISTA CLINICO,NAO ACIDENTE,ELETIVA,0.0,QUIMIOTERAPIA,NaN,C50,PROCEDIMENTOS,2.0,NEGADO,57
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
398384,GUIA DE SOLICITACAO SP/SADT,2439288.0,CLINICA,MEDICO ONCOLOGISTA CLINICO,NAO ACIDENTE,ELETIVA,0.0,QUIMIOTERAPIA,NaN,C50,PROCEDIMENTOS,1.0,NEGADO,57
398851,GUIA DE SOLICITACAO SP/SADT,2430846.0,CLINICA,MEDICO ONCOLOGISTA CLINICO,NAO ACIDENTE,ELETIVA,0.0,QUIMIOTERAPIA,NaN,C50,MAT/MED,100.0,AUTORIZADO,80
403104,GUIA DE SOLICITACAO SP/SADT,2430846.0,CLINICA,MEDICO ONCOLOGISTA CLINICO,NAO ACIDENTE,ELETIVA,0.0,QUIMIOTERAPIA,NaN,C50,MAT/MED,6.0,AUTORIZADO,80
408728,GUIA DE SOLICITACAO SP/SADT,2443007.0,CLINICA,MEDICO ONCOLOGISTA CLINICO,NAO ACIDENTE,ELETIVA,0.0,QUIMIOTERAPIA,NaN,C50,PROCEDIMENTOS,1.0,AUTORIZADO,47


In [47]:
conditions = {
    'GLAUCOMA': 'H40',
    'GLAUCOMA ': 'H40',
    'CD DE MAMA|C50': 'C50',
    'COVID(-19| 19)?': 'B34',
    'Z00': 'Z00',
    'ROTINA ': 'Z10',
    'I10': 'I10',
    'PACIENTE POSITIVO PARA COVID 19 HA 10 DIAS COM QUEIXA DE FEBRE, CEFALEIA E DESCONFORTO RESPIRATORIO, DOR TORACICA, REFERE DISPNEIA. PACIENTE REFRATARIO AO TRATAMENTO AMBULATORIAL COM PIORA DO QUADRO E MANUTENC?O DA FEBRE APOS USO DE ANABEOTERAPIA ORAL. PIORA DO PADR?O RADIOLOGICO COM TOMOGRAFIA APARENTANDO LES?ES <50% DE COMPROMETIMENTO': 'B34',
    'CID K21': 'k21',
    'CID: H 40.1 GLAUCOMA PRIMARIO  /  H18.5 DISTROFIA CORNEANA': 'H40',
    'RASTREAMENTO DE MAMAS PARA IDADE': 'Z12',
    'H40 GLAUCOMA ': 'H40',
    'H40 GLAUCOMA': 'H40',
    'SUSPEITA DE GLAUCOMA\.? ': 'H40',
    'CID: H 25.1 CATARATA SENIL NUCLEAR \+ H 36.0 RETINOPATIA DIABETICA': 'H25',
    'CATARATA': 'H25',
    'CATARATA EM OLHO DIREITO ':'H25',
    'CATARATA EM OLHO DIREITO':'H25',
    'CATARATA EM OLHO ESQUERDO ':'H25',
    'CATARATA EM OLHO DIREITO':'H25',
    'DRCT': 'N18',
    'CA DE COLON|C18': 'C18',
    'NEOPLASIA MALIGNA DA PROSTATA':'C61', 
    'NEOPLASIA DE MAMA': 'C50',
    'CA DE PROSTATA': 'C61',
    'IRA': 'C17',
    'CERATOCONE': 'H186',
    'CA DE OVARIO': 'D39',
    'CATARATA SENIL': 'H250',
    'PRE OPERATORIO DE CATARATA': 'H26',
    'R10 R11 K27': 'K27',
    'MIOPIA': 'H521',
    'HPB': 'N40',
    'DOR': 'R52',
    'C56': 'C56',
    'HAS': 'I10',
    'RASTREAMENTO' :'Z12',
    'C56' : 'C56',
    'CATARATA AO' : 'H25',
    'DOENCA CORIORRETINIANA': 'H30',
    'CA DE RETO': 'C20',
    'TONOMETRIA DE APLANAC?O E MAPEAMENTO DE RETINA S?O EXAMES COMPLEMENTARES A CONSULTA OFTALMOLOGICA': 'H33',
    'C20': 'C20',
    'C61' : 'C61',
    'C22': 'C22',
    'C49' : 'C49',
    'C49' : 'C49',
    'PRE NATAL': 'Z36',
    'C34': 'C34',
    'C90': 'C90',
    'GUIA AUTORIZADA E CONFIRMADA NO SISTEMA ANTIGO, CONVENIO ORIENTOU A SOLICITAR NOVAMENTE SOMENTE PARA QUEST?O DE FATURAMENTO.': 'DUPLICADOS',
    'D46': 'D46',
    'DISLIPIDEMIA': 'E78',
    'C16': 'C16',
    'MANUTENC?O DE CATETER': 'Z45',
    'SEM' : 'Z00',
    'H40.0': 'H40', 
    'C44': 'C44',
    'RINITE ALERGICA': 'J30',
    'ESTEATOSE HEPATICA': 'K79',
    'C25': 'C25',
    'Z00   EXAME GERAL INVEST PESS S/QUEIX DIAGN RELAT':'Z00',
    'SEPSE': 'A41',
    'COLELITIASE' : 'K80',
    'INFECCAO POR CORONAVIRUS DE LOCALIZACAO NAO ESPECIFICADA': 'B34',
    'COVID 19+': 'B34',
    'CA DE RIM': 'C64',
    'CA DE PULM?O' : 'C34',
    'DISFONIA': 'R49',
    'C43': 'C43' 
    
}

# Aplicando as condições de imputação
for pattern, cid in conditions.items():
    df.loc[df['DS_INDICACAO_CLINICA'].str.contains(pattern, case=False, na=False), 'CD_CID'] = cid

# Removendo linhas com 'DUPLICADOS'
df = df[df['CD_CID'] != "DUPLICADOS"]


C:\Users\moura\AppData\Local\Temp\ipykernel_8668\3048887933.py:77: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  df.loc[df['DS_INDICACAO_CLINICA'].str.contains(pattern, case=False, na=False), 'CD_CID'] = cid


In [48]:
# Verificando se mudou os números de nulos
nulos = df['CD_CID'].isna().sum()
nulos_ant = int(235294)

print(f"Agora temos", nulos, f"valores nulos. Foram {nulos_ant-nulos} dados imputados.")


Agora temos 163498 valores nulos. Foram 71796 dados imputados.


# k-means


Apesar do K-means ter elimiando todos os dados nulos, percebemos que essa não é a melhor estratégia, pois, usando uma tecnica de imputação de dados de agrupamentos pode excluir fenomenos mais raros, que parece ser o caso de doenças. 
Porém, vamos usar essa estratégia por enquanto. 

Agora vamos usar um método de clusterização para imputar o restante dos dados faltantes em CD_CID

# Terceiro Pré-processamento qu

Usando DS_CBOcpara tentar imputar dados em CD_cid a partir do k-means


In [49]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()

df['DS_CBO_ENCONDED'] = le.fit_transform(df['DS_CBO'])


In [50]:
# Colunas que são relevantes para CD_CID
columns_for_clustering = ['DS_CBO_ENCONDED', 'IDADE', 'QT_DIA_SOLICITADO']

# Imputando média nos valores ausentes (mas não tem valores ausentes)
imputer = SimpleImputer(strategy='mean')
df_clustering = imputer.fit_transform(df[columns_for_clustering])

# Normalizando os dados
scaler = StandardScaler()
df_clustering_scaled = scaler.fit_transform(df_clustering)

# K-means
kmeans = KMeans(n_clusters=10)
kmeans.fit(df_clustering_scaled)
clusters = kmeans.labels_
df['Cluster'] = clusters

# Imputando valores ausentes de CD_CID com base nos clusters
for cluster in df['Cluster'].unique():
    mode_value = df[df['Cluster'] == cluster]['CD_CID'].mode()[0]
    df.loc[(df['Cluster'] == cluster) & (df['CD_CID'].isna()), 'CD_CID'] = mode_value


c:\Users\moura\Anaconda3\envs\data_mining\lib\site-packages\sklearn\cluster\_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


In [51]:
nulos = df['CD_CID'].isna().sum()

print(nulos)

0


Agora consegimos perceber que o k-means conseguiu generalizar e aplicar os CID mais comuns para exames de determinadas especializações médicas. 
Não é a forma mais certeira de fazer imputação, mas pelo menos conseguimos tirar um pouco do enviesamento e não perder muitos dados

In [52]:
visualizacao = df.loc[df['DS_INDICACAO_CLINICA'] == 'CERATITE COM BAIXA AQUIDADE VISUAL ']
visualizacao

,DS_TIPO_GUIA,DT_NASCIMENTO,DS_TIPO_PREST_SOLICITANTE,DS_CBO,DS_INDICACAO_ACIDENTE,DS_CARATER_ATENDIMENTO,QT_DIA_SOLICITADO,DS_TIPO_ATENDIMENTO,CD_CID,DS_INDICACAO_CLINICA,DS_TIPO_ITEM,QT_SOLICITADA,DS_STATUS_ITEM,IDADE,DS_CBO_ENCONDED,Cluster
133028,GUIA DE SOLICITACAO SP/SADT,2436292.0,CLINICA,MEDICO OFTALMOLOGISTA,NAO ACIDENTE,ELETIVA,0.0,EXAMES,H16,CERATITE COM BAIXA AQUIDADE VISUAL,PROCEDIMENTOS,2.0,NEGADO,65,49,0
141041,GUIA DE SOLICITACAO SP/SADT,2436292.0,CLINICA,MEDICO OFTALMOLOGISTA,NAO ACIDENTE,ELETIVA,0.0,EXAMES,H16,CERATITE COM BAIXA AQUIDADE VISUAL,PROCEDIMENTOS,2.0,NEGADO,65,49,0
146214,GUIA DE SOLICITACAO SP/SADT,2436292.0,CLINICA,MEDICO OFTALMOLOGISTA,NAO ACIDENTE,ELETIVA,0.0,EXAMES,H16,CERATITE COM BAIXA AQUIDADE VISUAL,PROCEDIMENTOS,2.0,NEGADO,65,49,0
170575,GUIA DE SOLICITACAO SP/SADT,2436292.0,CLINICA,MEDICO OFTALMOLOGISTA,NAO ACIDENTE,ELETIVA,0.0,EXAMES,H16,CERATITE COM BAIXA AQUIDADE VISUAL,PROCEDIMENTOS,2.0,NEGADO,65,49,0


In [53]:
# Podemos remover a cluna DT_NASCIMENTO agora que temos a IDADE
df = df.drop(columns='DT_NASCIMENTO', errors='ignore')
df = df.drop(columns='DS_INDICACAO_CLINICA', errors='ignore')

In [55]:
'''df = df[['DS_TIPO_GUIA', 'IDADE', 
         'DS_TIPO_PREST_SOLICITANTE', 'DS_CBO', 'DS_CBO_ENCONDED', 
         'DS_INDICACAO_ACIDENTE', 'DS_CARATER_ATENDIMENTO', 'QT_DIA_SOLICITADO',
         'DS_TIPO_ATENDIMENTO','CD_CID', 
         'DS_TIPO_ITEM', 'QT_SOLICITADA'
        ]]'''

df = df[['DS_TIPO_GUIA', 'IDADE', 
         'DS_TIPO_PREST_SOLICITANTE', 'DS_CBO', 'DS_CBO_ENCONDED', 
         'DS_INDICACAO_ACIDENTE', 'DS_CARATER_ATENDIMENTO', 'QT_DIA_SOLICITADO',
         'DS_TIPO_ATENDIMENTO','CD_CID', 
         'DS_TIPO_ITEM', 'QT_SOLICITADA', 'DS_STATUS_ITEM'
        ]]

df.head()

,DS_TIPO_GUIA,IDADE,DS_TIPO_PREST_SOLICITANTE,DS_CBO,DS_CBO_ENCONDED,DS_INDICACAO_ACIDENTE,DS_CARATER_ATENDIMENTO,QT_DIA_SOLICITADO,DS_TIPO_ATENDIMENTO,CD_CID,DS_TIPO_ITEM,QT_SOLICITADA,DS_STATUS_ITEM
0,GUIA DE SOLICITACAO SP/SADT,54,HOSPITAL,MEDICO ORTOPEDISTA E TRAUMATOLOGISTA,51,NAO ACIDENTE,URGENCIA/EMERGENCIA,0.0,PRONTO SOCORRO,R52,PROCEDIMENTOS,1.0,NEGADO
1,GUIA DE SOLICITACAO SP/SADT,57,PRESTADOR DE SERVICOS,MEDICO DERMATOLOGISTA,28,NAO ACIDENTE,ELETIVA,0.0,PEQUENA CIRURGIA,Z00,PROCEDIMENTOS,1.0,AUTORIZADO
2,GUIA DE SOLICITACAO INTERNACAO,45,HOSPITAL,MEDICO CLINICO,25,NAO ACIDENTE,URGENCIA/EMERGENCIA,3.0,INTERNACAO (SADT INTERNADO),B34,PROCEDIMENTOS,1.0,AUTORIZADO
3,GUIA DE SOLICITACAO SP/SADT,84,CLINICA,MEDICO CARDIOLOGISTA,16,NAO ACIDENTE,ELETIVA,0.0,EXAMES,I10,PROCEDIMENTOS,1.0,AUTORIZADO
4,GUIA DE SOLICITACAO SP/SADT,56,CLINICA,MEDICO ORTOPEDISTA E TRAUMATOLOGISTA,51,NAO ACIDENTE,ELETIVA,0.0,PEQUENA CIRURGIA,R52,MAT/MED,1.0,AUTORIZADO


In [ ]:
# SALVANDO O DATASET NOVO SEM VALORES NULOS

# o pre_processamento_3 foi usado um k-means com mais colunas para clusteziração, principalmnte DS_CBO que contem as
# especializações médicas que por sua vez ajuda na clusterização mais adequada de acordo com a especializade do médico solicitante

df.to_csv('pre_processamento_3_test.csv', index=False, encoding='utf-8', sep=',')
